In [1]:
# generate the symmetry-corrected indices for move-based convolution
import sys
import numpy as np
sys.path.append('..')

In [2]:
# test the coefficient generation logic the naive way
cell = 22

In [3]:
# display-only code, to visually check the coeffs are in the correct locations on the board
from neural.neural_ import to_pair, generate_all_moves_by_index, move_convolution_indices

all_inds, num_coeffs = move_convolution_indices()
num_coeffs -= 10 # the first 10 in the above function are biases, don't need them
num_biases = 10
num_fields = 7*7

cell = cell+1
tmp = all_inds[cell]
a = np.zeros([7,7])
for (ind, coeff) in tmp[1:]:
    pair = to_pair(ind)
    a[pair[0],pair[1]] = coeff - 9

print(to_pair(cell))
print(a)

(2, 3)
[[  0.   0.  39.   0.  39.   0.   0.]
 [  0.  37.   0.   0.   0.  37.   0.]
 [  0.   0.   0.   9.   0.   0.   0.]
 [  0.  36.   0.   0.   0.  36.   0.]
 [  0.   0.  38.   0.  38.   0.   0.]
 [  0.   0.   0.   0.   0.   0.   0.]
 [  0.   0.   0.   0.   0.   0.   0.]]


In [4]:
# try calling conv_stack directly from Tensorflow
import numpy as np
import tensorflow as tf


tf.reset_default_graph()

from neural.tensorflow_utils import conv_stack

with tf.Session() as sess:
    in_fields_np = np.ones([2,num_fields])
    in_fields_np[0,3] = 0
    in_fields_np[0,5] = 0
    my_pos = np.array([24, 24])
    other_pos =  np.array([33,33])
    inputs_np = np.concatenate([in_fields_np, my_pos[:,None], other_pos[:,None]],
                              1)
    #print(inputs_np.shape)
    inputs =tf.constant(inputs_np)# tf.placeholder(shape =[None, num_fields+2], dtype = tf.float32) #
    #print(sess.run(inputs))
    out = conv_stack(inputs, 5,sess)
    
    dummy = np.array([50,50])[:,None]
    #print(sess.run(get_random_index(inputs, tf.constant(dummy))))
    
    sess.run(tf.global_variables_initializer())
    stack_result = sess.run(out)#, feed_dict={inputs:inputs_np})
    print(stack_result)

[2 3 2]
[[[-15.65185452   1.91709328]
  [ 26.72372246   1.52010012]
  [ -6.86462402  -0.48497367]]

 [[-20.92775726   2.48585844]
  [ 33.23313141   0.52431291]
  [ -9.04220009  -0.66439515]]]


In [31]:
# load game simulation data
import glob
import sys
import pickle
from neural.data_utils import load_simulation_data

fn = '../data/states.pickle'
try:
    with open(fn, 'rb') as f:
        states = pickle.load(f)
except:
    files = glob.glob('../data/ID_x2_1000ms/result_ID*.pickle')
    #print(files)
    depths =load_simulation_data(files)
    keys = list(depths.keys())
    #print(keys)
    games = depths[keys[0]]
    #print(games[0])
    states = [state for game in games for state in game] 
    print(len(states))
    with open(fn, 'wb') as f:
        pickle.dump(states,f)

In [7]:
# attempt to fit a simple score

from keras.models import Model, Sequential
from keras.layers import Input, Lambda, Flatten, Dense, Activation
from keras.layers.merge import Concatenate, Add
from keras.layers.normalization import BatchNormalization
from keras import backend as K

from neural.keras_utils import ConvByMoveLayer

player_pos_one_hot = Input(shape = [49, 2])
board_state = Input(shape=[49,1])
mask = board_state
num_features = 4

def ResNetLayerFun(x, num_features = 3, mask = None):
    tmp = BatchNormalization()(x)
    tmp = Activation('relu')(tmp)
    tmp = ConvByMoveLayer(num_features, mask)(tmp)
    tmp = BatchNormalization()(tmp)
    tmp = Activation('relu')(tmp)
    tmp = ConvByMoveLayer(num_features, mask)(tmp)
    return Add()([x,tmp])

#tmp1 = K.expand_dims(board_state, 2)# TODO: do this in Keras code
out = Concatenate()([board_state, player_pos_one_hot])
out = ConvByMoveLayer(num_features, mask)(out)
out = ResNetLayerFun(out, num_features, mask)
out = ResNetLayerFun(out, num_features, mask)
out = Activation('relu')(out)
out = Concatenate()([out, player_pos_one_hot])
out = Flatten()(out)
out = Dense(10, activation = 'relu')(out)
out = Dense(1)(out)

model = Model(inputs = [player_pos_one_hot, board_state], outputs = out)
model.summary()
model.compile(optimizer = 'adam',  loss='mean_squared_error')

Using TensorFlow backend.


____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
input_2 (InputLayer)             (None, 49, 1)         0                                            
____________________________________________________________________________________________________
input_1 (InputLayer)             (None, 49, 2)         0                                            
____________________________________________________________________________________________________
concatenate_1 (Concatenate)      (None, 49, 3)         0           input_2[0][0]                    
                                                                   input_1[0][0]                    
____________________________________________________________________________________________________
conv_by_move_layer_1 (ConvByMove (None, 49, 4)         532         concatenate_1[0][0]     

In [28]:
from neural.data_utils import prepare_data_for_model
board_full, player_pos, y = prepare_data_for_model(states)
print(len(y))

984694


In [29]:
model.fit([player_pos, board_full],y, batch_size = 256, epochs=10, verbose =1)

Epoch 1/10
308992/984694 [========>.....................] - ETA: 70s - loss: 1.1580

KeyboardInterrupt: 

In [24]:
# Now let's get all those games where tree search actually completed
import numpy as np
from sklearn.preprocessing import OneHotEncoder

complete_states = [state for state in states if state['score'] == float('inf') or state['score'] == float('-inf')]
print(len(complete_states))
board_full_c, player_pos_c, y_c = prepare_data_for_model(complete_states,'score')
y_c[y_c==float('inf')] = 1
y_c[y_c==float('-inf')] = 0

4795


In [17]:
print(set(list(np.reshape(y_c,[-1]))))

{0.0, 1.0}


In [18]:
from keras.models import Model, Sequential
from keras.layers import Input, Lambda, Flatten, Dense, Activation, Dropout
from keras.layers.merge import Concatenate, Add
from keras.layers.normalization import BatchNormalization
from keras import backend as K

player_pos_one_hot = Input(shape = [49, 2])
board_state = Input(shape=[49,1])
mask = board_state
num_features = 16
num_res_modules = 16
drop_rate = 0.1

def ResNetLayerFun(x, num_features = 3, mask = None):
    tmp = BatchNormalization()(x)
    tmp = Activation('relu')(tmp)
    tmp = ConvByMoveLayer(num_features, mask)(tmp)
    tmp = BatchNormalization()(tmp)
    tmp = Activation('relu')(tmp)
    tmp = ConvByMoveLayer(num_features, mask)(tmp)
    tmp = Dropout(drop_rate)(tmp)
    return Add()([x,tmp])

#tmp1 = K.expand_dims(board_state, 2)# TODO: do this in Keras code
out = Concatenate()([board_state, player_pos_one_hot])
out = ConvByMoveLayer(num_features, mask)(out)
for _ in range(num_res_modules):
    out = ResNetLayerFun(out, num_features, mask)
out = Activation('relu')(out)
out = Concatenate()([out, player_pos_one_hot])
out = Flatten()(out)
out = Dense(10, activation = 'relu')(out)
out = Dense(2, activation = 'softmax')(out)

deep_model = Model(inputs = [player_pos_one_hot, board_state], outputs = out)
deep_model.summary()
deep_model.compile(optimizer = 'adam',  loss='categorical_crossentropy', metrics =['acc'])

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
input_6 (InputLayer)             (None, 49, 1)         0                                            
____________________________________________________________________________________________________
input_5 (InputLayer)             (None, 49, 2)         0                                            
____________________________________________________________________________________________________
concatenate_5 (Concatenate)      (None, 49, 3)         0           input_6[0][0]                    
                                                                   input_5[0][0]                    
____________________________________________________________________________________________________
conv_by_move_layer_39 (ConvByMov (None, 49, 16)        2128        concatenate_5[0][0]     

In [25]:
from keras.utils.np_utils import to_categorical

deep_model.fit([player_pos_c, board_full_c],to_categorical(y_c, num_classes=2), batch_size = 256, epochs=10, verbose =1, validation_split = 0.1)

Train on 4315 samples, validate on 480 samples
Epoch 1/10
4315/4315 [==============================] - 4s - loss: 0.2558 - acc: 0.9041 - val_loss: 0.2792 - val_acc: 0.8896
Epoch 2/10
4315/4315 [==============================] - 3s - loss: 0.2307 - acc: 0.9131 - val_loss: 0.2171 - val_acc: 0.9250
Epoch 3/10
4315/4315 [==============================] - 3s - loss: 0.2055 - acc: 0.9196 - val_loss: 0.2476 - val_acc: 0.9229
Epoch 4/10
4315/4315 [==============================] - 3s - loss: 0.1828 - acc: 0.9289 - val_loss: 0.2723 - val_acc: 0.9062
Epoch 5/10
4315/4315 [==============================] - 3s - loss: 0.1686 - acc: 0.9351 - val_loss: 0.2534 - val_acc: 0.9021
Epoch 6/10
4315/4315 [==============================] - 3s - loss: 0.1605 - acc: 0.9363 - val_loss: 0.2563 - val_acc: 0.9021
Epoch 7/10
4315/4315 [==============================] - 4s - loss: 0.1569 - acc: 0.9353 - val_loss: 0.3352 - val_acc: 0.8917
Epoch 8/10
4315/4315 [==============================] - 3s - loss: 0.1340 - ac

In [ ]:
from collections import namedtuple
from copy import copy

# 'pos' is [pos of player about to move, other_pos]
# 'game' is a vector of 0s for used fields, 1s for available fields

#SimpleGame = namedtuple("Simple_game", ["moving_player_pos","other_player_pos", "board"])
move_dict = generate_all_moves_by_index()

def get_legal_moves(game):
    if game['pos'][0] is None:
        return [m for m in range(49) if game['game'][m] == 1]
    else:
        moves = move_dict[game['pos'][0]]
        return [m for m in moves if game['game'][m] == 1]

def apply_move(game, move):
    if not move in get_legal_moves(game):
        raise ValueError('Illegal move!')
    new_board = copy(game['game'])
    new_board[move] = 0
    other_pos = move
    moving_pos = game['pos'][1]
    return {'game': new_board, 'pos': np.array([moving_pos, other_pos])}

def get_best_move_from_model(game, model = deep_model):
    moves = get_legal_moves(game)
    tmp = [apply_move(game, move) for move in moves]
    board, pos, _ = prepare_data_for_model(tmp,None)
    valuations = model.predict([pos, board])[:,0]
    best_ind = np.argmax(valuations)
    #print(best_ind)
    return moves[best_ind], valuations[best_ind]
        
    
    
board = np.ones(49)
print(list(board))
board.sum()
my_pos = None
other_pos = None
game = {'pos': np.array([my_pos, other_pos]), 'game': board}
game1 = apply_move(game, 0)
game2 = apply_move(game1, 1)
print(get_legal_moves(game2))
game3 = apply_move(game2, 15)
# board, pos, _ = possible_moves_for_model(game3)
# print(board.shape, pos.shape)
get_best_move_from_model(game3)


In [ ]:
# sort all games by number of moves. 
states_by_num_moves = [[] for _ in range(49)]

for state in states:
    moves_made = 49 - state['game'].sum()
    states_by_num_moves[int(moves_made)].append(state)
    
for n in range(49):
    print(n,len(states_by_num_moves[n]))

In [ ]:
# Iteratively populate all non-+-inf values in layer n from evaluating model in layer n+1, then include these into the fitting set
# after each pass, refresh the values for earlier layers
def recursively_fill_scores(states, model = deep_model):
    print(len(states))
    scores = np.zeros([len(states)])
    for n,state in enumerate(states):
        if state['score']  == float('inf'):
            scores[n] = 1
        elif state['score'] == float('-inf'):
            scores[n] = 0
        else:
            _ , scores[n] = get_best_move_from_model(state, model)
        if n%1000 == 0:
            print(n)
    return scores

prepared_data = [None for _ in range(49)]

for n in range(18,49):
    if len(states_by_num_moves[n]):
        board, pos, _  = prepare_data_for_model( states_by_num_moves[n], None) # board, player_pos, score
        scores = recursively_fill_scores( states_by_num_moves[n])
        prepared_data[n] = (pos, board, scores)
        print(len(scores),len(set(list(scores))))
# TODO: is my position always first in those dumps???
    

            


            
